# Notebook for CAV2020 Artifact Evaluation - Submission 233

This notebook contains the tests to reproduce the results presented in our paper titled *Qualitative Controller Synthesis for Consumption Markov Decision Processes*. It also helps understand the package structure for ease of reusability.

Please note that the computation time for different tasks reported in the paper change based on the machine configuration but the scalabilty remains the same. The following computation times are obtained using a machine with the Intel® Core™ i7-8700 CPU @ 3.20GHz × 12 processor and a RAM of 16 GB running Ubuntu 18.04 LTS. This is a better configuration than the portable laptop used to generate the results reported in the paper and hence the computation times are better. 

We provide detailed documentation and installation instructions using the documentation served using ReadtheDocs at the following link: [documentation](https://fimdp.readthedocs.io/en/latest/).

While all the tests below are self-contained, we encourage you to have a look at other example notebooks. You can find brief descriptions of other example notebooks in the readme file in the current directory or at this [link](https://fimdp.readthedocs.io/en/latest/examples.html).

We consider the following two case-studies in our paper which model the problems of an electric vehicle routing with limited capacity and a multi-agent grid world inspired by the Mars 2020 mission. For detailed description of the problems, refer to the **Implementation and Csse Studies** section of the paper or the following links:

* Description of Electric Vehicle Routing Problem: [link](https://fimdp.readthedocs.io/en/latest/examples.html#electric-vehicle-routing)
* Description of Multi-agent Grid World Problem: [link](https://fimdp.readthedocs.io/en/latest/examples.html#multi-agent-grid-world)

The following two subsections reproduce the results from the corresponding subsections in the **Implementation and Csse Studies** section of the paper.


## Electric Vehicle Routing

This subsection reproduces the results in section *7.1 Electric Vehicle Routing* of the paper. We begin with importing required modules from fimdp.

In [1]:
import sys; sys.path.insert(0, '..')
import fimdp.nyc_parser as nyc_parser
import fimdp.NYCtools as NYCtools

We generate the consMDP object representing the consumption MDP modeling the problem of electric vehicle routing in NYC. NYCstreetnetwork.json contains a stochastic energy consumption model covering all the streets in central Manhattan, New York City.

In [2]:
m, targets = nyc_parser.parse('NYCstreetnetwork.json')

### Mean computation time for different objectives

Mean computation time for *Safe* objective (safety) for an energy capacity of 100.

In [3]:
%%timeit
m.get_safe(100, recompute=True)

1.07 s ± 14.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Mean computation time for *SafePR_T* (postive reachability) objective for an energy capacity of 100.

In [14]:
%%timeit
m.get_positiveReachability(targets, 100, recompute=True)

1.39 s ± 29.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Mean computation time for *SafeBüchi_T* (Büchi) objective for an energy capacity of 100.

In [16]:
%%timeit
m.get_Buchi(targets, 100, recompute=True)

1.42 s ± 48.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Mean computation time for different capacities

We now analyze the variation of computation time for different capacities. The task is to generate a strategy with Büchi objective. These results (on a different config) were reported in Table 1 of the paper and show a similar trend.

**Capacity = 3**

In [19]:
%%timeit
m.get_Buchi(targets, 3, recompute=True)

126 ms ± 4.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


**Capacity = 5**

In [21]:
%%timeit
m.get_Buchi(targets, 5, recompute=True)

253 ms ± 8.27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Capacity = 7**

In [22]:
%%timeit
m.get_Buchi(targets, 7, recompute=True)

406 ms ± 8.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Capacity = 10**

In [23]:
%%timeit
m.get_Buchi(targets, 10, recompute=True)

608 ms ± 11.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Capacity = 15**

In [24]:
%%timeit
m.get_Buchi(targets, 15, recompute=True)

700 ms ± 14.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Capacity = 20**

In [25]:
%%timeit
m.get_Buchi(targets, 20, recompute=True)

1.24 s ± 15.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Capacity = 100**

In [26]:
%%timeit
m.get_Buchi(targets, 100, recompute=True)

1.44 s ± 27.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Multi-agent Grid World

This subsection reproduces the results in section *7.2 Multi-agent Grid World* of the paper. We begin with importing required modules from fimdp. **Note that an environment with a grid size $n$ has $n^4$ states in the corresponding Consumption MDP.**

In [5]:
import sys; sys.path.insert(0, '..')
from fimdp.MarsEnv import MarsEnv

### Mean computation time for different capacities

We begin by creating a environment with grid size $n = 10$ and agent capacity of 10 units.

In [28]:
env = MarsEnv(grid_size=10, agent_capacity=100, agent_actioncost=1, agent_staycost=1)
m, targets = env.get_mdp_targets()

Mean computation time for *Safe* objective (safety) for an energy capacity of 100.

In [29]:
%%timeit
m.get_safe(env.agent_capacity, recompute=True)

13.1 s ± 143 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Mean computation time for *SafePR_T* (postive reachability) objective for an energy capacity of 100.

In [30]:
%%timeit
m.get_positiveReachability(targets, env.agent_capacity, recompute=True)

11.5 s ± 132 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Mean computation time for *SafeBüchi_T* (Büchi) objective for an energy capacity of 100.

In [ ]:
%%timeit
m.get_Buchi(targets, env.agent_capacity, recompute=True)

### Mean computation time for Different Grid Sizes

We calculate the computation time for obtaining the strategy for different sizes of grid (different sizes of state spaces) for the same objective and agent capacity. These results were reported in Table 2 in the paper. We consider the Büchi objective and a capacity of 10 for our tests. We note that the absolute values of the computation time vary on the machine used but should have the same trend in scalability.

In [32]:
from fimdp.energy_solver import *

* **Grid size of $n = 5$, i.e., $625$ states.**

In [33]:
%%timeit
env = MarsEnv(grid_size=5, agent_capacity=10, agent_actioncost=1, agent_staycost=1)
m, targets = env.get_mdp_targets()
s = EnergySolver(m, cap=env.agent_capacity, targets=targets)
strategy = s.get_strategy(BUCHI, recompute=True)

707 ms ± 30.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


* **Grid size $n = 7$, i.e., $2401$ states.**

In [14]:
%%timeit
env = MarsEnv(grid_size=7, agent_capacity=10, agent_actioncost=1, agent_staycost=1)
m, targets = env.get_mdp_targets()
s = EnergySolver(m, cap=env.agent_capacity, targets=targets)
strategy = s.get_strategy(BUCHI, recompute=True)

3.21 s ± 80.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


* **Grid size $n = 10$, i.e., $10000$ states.**

In [16]:
%%timeit
env = MarsEnv(grid_size=10, agent_capacity=10, agent_actioncost=1, agent_staycost=1)
m, targets = env.get_mdp_targets()
s = EnergySolver(m, cap=env.agent_capacity, targets=targets)
strategy = s.get_strategy(BUCHI, recompute=True)

14.8 s ± 165 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


* **Grid size $n = 15$, i.e., $50625$ states.**

In [17]:
%%timeit
env = MarsEnv(grid_size=15, agent_capacity=10, agent_actioncost=1, agent_staycost=1)
m, targets = env.get_mdp_targets()
s = EnergySolver(m, cap=env.agent_capacity, targets=targets)
strategy = s.get_strategy(BUCHI, recompute=True)

1min 23s ± 141 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


* **Grid size $n = 20$, i.e., $160000$ states.**

In [20]:
%%timeit
env = MarsEnv(grid_size=20, agent_capacity=10, agent_actioncost=1, agent_staycost=1)
m, targets = env.get_mdp_targets()
s = EnergySolver(m, cap=env.agent_capacity, targets=targets)
strategy = s.get_strategy(BUCHI, recompute=True)

4min 55s ± 6.12 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
